<a href="https://colab.research.google.com/github/Lucia-Garcia-Lado/Graph-Coloring-Models-for-Production-Line-Scheduling-Optimization/blob/main/graph_coloring_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from deap import base, creator, tools, algorithms
import random

In [4]:
# ------------------------------------------------------------
# 1. Problema: FT06 (6 Trabajos, 6 Máquinas)
# ------------------------------------------------------------
# Makespan Óptimo (conocido): 55

OPERATIONS = [
    # --- Job 0 ---
    {"job": 0, "machine": 2, "duration": 1, "order": 0},
    {"job": 0, "machine": 0, "duration": 3, "order": 1},
    {"job": 0, "machine": 1, "duration": 6, "order": 2},
    {"job": 0, "machine": 3, "duration": 7, "order": 3},
    {"job": 0, "machine": 5, "duration": 3, "order": 4},
    {"job": 0, "machine": 4, "duration": 6, "order": 5},

    # --- Job 1 ---
    {"job": 1, "machine": 1, "duration": 8, "order": 0},
    {"job": 1, "machine": 2, "duration": 5, "order": 1},
    {"job": 1, "machine": 4, "duration": 10, "order": 2},
    {"job": 1, "machine": 5, "duration": 10, "order": 3},
    {"job": 1, "machine": 0, "duration": 10, "order": 4},
    {"job": 1, "machine": 3, "duration": 4, "order": 5},

    # --- Job 2 ---
    {"job": 2, "machine": 2, "duration": 5, "order": 0},
    {"job": 2, "machine": 3, "duration": 4, "order": 1},
    {"job": 2, "machine": 5, "duration": 8, "order": 2},
    {"job": 2, "machine": 0, "duration": 9, "order": 3},
    {"job": 2, "machine": 1, "duration": 1, "order": 4},
    {"job": 2, "machine": 4, "duration": 7, "order": 5},

    # --- Job 3 ---
    {"job": 3, "machine": 1, "duration": 5, "order": 0},
    {"job": 3, "machine": 0, "duration": 5, "order": 1},
    {"job": 3, "machine": 2, "duration": 5, "order": 2},
    {"job": 3, "machine": 3, "duration": 3, "order": 3},
    {"job": 3, "machine": 4, "duration": 8, "order": 4},
    {"job": 3, "machine": 5, "duration": 9, "order": 5},

    # --- Job 4 ---
    {"job": 4, "machine": 2, "duration": 9, "order": 0},
    {"job": 4, "machine": 1, "duration": 3, "order": 1},
    {"job": 4, "machine": 4, "duration": 5, "order": 2},
    {"job": 4, "machine": 5, "duration": 4, "order": 3},
    {"job": 4, "machine": 0, "duration": 3, "order": 4},
    {"job": 4, "machine": 3, "duration": 1, "order": 5},

    # --- Job 5 ---
    {"job": 5, "machine": 1, "duration": 3, "order": 0},
    {"job": 5, "machine": 3, "duration": 3, "order": 1},
    {"job": 5, "machine": 5, "duration": 9, "order": 2},
    {"job": 5, "machine": 0, "duration": 10, "order": 3},
    {"job": 5, "machine": 4, "duration": 4, "order": 4},
    {"job": 5, "machine": 2, "duration": 1, "order": 5},
]

NUM_OPS = len(OPERATIONS)
HORIZON = 300

In [5]:
# 1. Mapa ID Máquina -> Lista de (índice, operación)
MACHINE_MAP = {}
machine_ids = set(op["machine"] for op in OPERATIONS)
for m in machine_ids:
    MACHINE_MAP[m] = [(i, op) for i, op in enumerate(OPERATIONS) if op["machine"] == m]

# 2. Mapa ID Trabajo -> Lista de (índice, operación) ordenadas por secuencia
JOB_MAP = {}
job_ids = set(op["job"] for op in OPERATIONS)
for j in job_ids:
    ops = [(i, op) for i, op in enumerate(OPERATIONS) if op["job"] == j]
    ops.sort(key=lambda x: x[1]["order"])
    JOB_MAP[j] = ops

# ------------------------------------------------------------
# 3. Función de Fitness (Evaluación)
# ------------------------------------------------------------

def evaluate(ind):

    # --- CONFLICTOS DE MÁQUINA ---
    machine_conflicts = 0

    for machine_id, machine_ops in MACHINE_MAP.items():

        # Comparamos cada par de operaciones en la misma máquina
        for i in range(len(machine_ops)):
            for j in range(i + 1, len(machine_ops)):

                index_A, op_data_A = machine_ops[i]
                index_B, op_data_B = machine_ops[j]

                start_A = ind[index_A]
                end_A = start_A + op_data_A["duration"]

                start_B = ind[index_B]
                end_B = start_B + op_data_B["duration"]

                # Comprobamos el overlap
                if start_A < end_B and start_B < end_A:
                    machine_conflicts += 1

    # --- CONFLICTOS DE PRECEDENCIA ---
    precedence_conflicts = 0

    for job_id, job_steps in JOB_MAP.items():

        # Iteramos sobre los pasos secuenciales
        for k in range(len(job_steps) - 1):
            current_step_idx, current_step_data = job_steps[k]
            next_step_idx, next_step_data = job_steps[k+1]

            current_step_end_time = ind[current_step_idx] + current_step_data["duration"]
            next_step_start_time = ind[next_step_idx]

            # El siguiente paso no puede empezar antes de que termine el actual
            if next_step_start_time < current_step_end_time:
                precedence_conflicts += 1

    # --- OBJETIVOS ---
    end_times = [ind[i] + OPERATIONS[i]["duration"] for i in range(NUM_OPS)]
    Cmax = max(end_times) if end_times else 0

    # Energía (Peak Load)
    events = []
    for i in range(NUM_OPS):
        start = ind[i]
        end = start + OPERATIONS[i]["duration"]
        events.append((start, 1))   # Máquina ON
        events.append((end, -1))    # Máquina OFF

    events.sort()

    peak_load = 0
    current_load = 0
    for _, change in events:
        current_load += change
        if current_load > peak_load:
            peak_load = current_load

    energy = peak_load

    # --- SUMA PONDERADA ---
    alpha = 1000  # Penalización alta por conflicto de máquina
    beta = 1000   # Penalización alta por conflicto de precedencia
    gamma = 10    # Peso para Carga Pico
    delta = 0.1   # Peso para Makespan

    total_cost = (alpha * machine_conflicts) + \
                 (beta * precedence_conflicts) + \
                 (gamma * energy) + \
                 (delta * Cmax)

    return total_cost,

In [6]:
# ------------------------------------------------------------
# 4. Operadores Genéticos
# ------------------------------------------------------------

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Generación de genes e individuos
toolbox.register("gene", random.randint, 0, HORIZON)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gene, NUM_OPS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Operadores estándar
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxTwoPoint)

def shift_mutation(ind, max_shift=10):
    idx = random.randrange(NUM_OPS)
    ind[idx] += random.randint(-max_shift, max_shift)
    # Mantener dentro de los límites válidos [0, HORIZON]
    ind[idx] = max(0, min(HORIZON, ind[idx]))
    return ind,

toolbox.register("mutate", shift_mutation)

In [7]:
# ------------------------------------------------------------
# 5. Bucle Principal del Algoritmo Genético
# ------------------------------------------------------------

def main():

    pop_size = 200
    ngen = 300
    cxpb = 0.8
    mutpb = 0.3

    pop = toolbox.population(n=pop_size)

    # Evaluamos la población inicial
    for ind in pop:
        ind.fitness.values = toolbox.evaluate(ind)

    print(f"Iniciando evolución... (Horizonte={HORIZON}, Pob={pop_size})")

    for gen in range(ngen):

        # Generar descendencia
        offspring = algorithms.varAnd(pop, toolbox, cxpb, mutpb)

        # Evaluar descendencia
        for ind in offspring:
            ind.fitness.values = toolbox.evaluate(ind)

        # Seleccionamos de AMBOS (población actual + descendencia) para asegurar elitismo.
        pop = toolbox.select(pop + offspring, k=pop_size)

        best = tools.selBest(pop, k=1)[0]

        # Imprimimos solo cada 10 generaciones para evitar llenar la consola
        if gen % 10 == 0:
            print(f"Gen {gen:3d} | Mejor coste: {best.fitness.values[0]:.2f}")

    # Resultados Finales
    best_final = tools.selBest(pop, k=1)[0]
    print("\n--- Resultado Final ---")
    print(f"Mejor coste: {best_final.fitness.values[0]:.2f}")
    print(f"Horario (Start Times): {best_final}")

    cost_val = best_final.fitness.values[0]
    if cost_val < 1000:
        print("La solución es FACTIBLE")
    else:
        print("La solución es INFACTIBLE (Penalizaciones > 0)")

if __name__ == "__main__":
    main()

Iniciando evolución... (Horizonte=300, Pob=200)
Gen   0 | Mejor coste: 12060.30
Gen  10 | Mejor coste: 7050.30
Gen  20 | Mejor coste: 4050.30
Gen  30 | Mejor coste: 4049.40
Gen  40 | Mejor coste: 4049.20
Gen  50 | Mejor coste: 4049.20
Gen  60 | Mejor coste: 4049.20
Gen  70 | Mejor coste: 4049.10
Gen  80 | Mejor coste: 4049.10
Gen  90 | Mejor coste: 4049.10
Gen 100 | Mejor coste: 4048.90
Gen 110 | Mejor coste: 4048.90
Gen 120 | Mejor coste: 4048.90
Gen 130 | Mejor coste: 4048.90
Gen 140 | Mejor coste: 4048.90
Gen 150 | Mejor coste: 4048.90
Gen 160 | Mejor coste: 4048.80
Gen 170 | Mejor coste: 4048.80
Gen 180 | Mejor coste: 4048.80
Gen 190 | Mejor coste: 4048.80
Gen 200 | Mejor coste: 4048.80
Gen 210 | Mejor coste: 4048.80
Gen 220 | Mejor coste: 4048.80
Gen 230 | Mejor coste: 4048.80
Gen 240 | Mejor coste: 4048.80
Gen 250 | Mejor coste: 4048.80
Gen 260 | Mejor coste: 4048.80
Gen 270 | Mejor coste: 4048.80
Gen 280 | Mejor coste: 4048.80
Gen 290 | Mejor coste: 4048.80

--- Resultado Final 